In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/BTCV

/content/drive/MyDrive/BTCV


In [4]:
!pwd

/content/drive/MyDrive/BTCV


In [5]:
pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.1/74.1 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.8/308.8 KB 29.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 88.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [14]:
import torch
import numpy as np
import os
import re
import nibabel as nib

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

organs = {1: "spleen",
          2: "right kidney",
          3: "left kidney",
          4: "gallbladder",
          5: "esophagus",
          6: "liver",
          7: "stomach",
          8: "aorta",
          9: "inferior vena cava",
          10: "portal vein and splenic vein",
          11: "pancreas",
          12: "right adrenal gland",
          13: "left adrenal gland"}

train_path = os.path.join("Abdomen", "RegData", "Training-Training", "img")
test_path = os.path.join("Abdomen", "RegData", "Training-Testing", "img")
train_dict = {}
valid_dict = {}
test_dict = {}

for dir in os.listdir(os.path.join("./dataset", train_path)):
    for img in os.listdir(os.path.join("./dataset", train_path, dir)):
        nii_label = nib.load(os.path.join("./dataset", train_path.replace("img", "label"), dir, img.replace("img", "label")))
        label_data = torch.tensor(nii_label.get_fdata())
        label_unique = torch.unique(label_data)
        if int(re.findall(r'\d+', img)[0]) < 35:
            train_dict[os.path.join("./dataset", train_path, dir, img)] = label_unique.type(torch.int8).tolist()
        else:
            valid_dict[os.path.join("./dataset", train_path, dir, img)] = list(label_unique)            

for dir in os.listdir(os.path.join("./dataset", test_path)):
    for img in os.listdir(os.path.join("./dataset", test_path, dir)):
        nii_label = nib.load(os.path.join("./dataset", test_path.replace("img", "label"), dir, img.replace("img", "label")))
        label_data = torch.tensor(nii_label.get_fdata())
        label_unique = torch.unique(label_data)
        test_dict[os.path.join("./dataset", test_path, dir, img)] = list(label_unique)

In [19]:
valid_dict

{'./dataset/Abdomen/RegData/Training-Training/img/0009/img0035-0009.nii.gz': [tensor(0., dtype=torch.float64),
  tensor(1., dtype=torch.float64),
  tensor(2., dtype=torch.float64),
  tensor(3., dtype=torch.float64),
  tensor(5., dtype=torch.float64),
  tensor(6., dtype=torch.float64),
  tensor(7., dtype=torch.float64),
  tensor(8., dtype=torch.float64),
  tensor(9., dtype=torch.float64),
  tensor(10., dtype=torch.float64),
  tensor(11., dtype=torch.float64),
  tensor(12., dtype=torch.float64),
  tensor(13., dtype=torch.float64)],
 './dataset/Abdomen/RegData/Training-Training/img/0009/img0036-0009.nii.gz': [tensor(0., dtype=torch.float64),
  tensor(1., dtype=torch.float64),
  tensor(2., dtype=torch.float64),
  tensor(3., dtype=torch.float64),
  tensor(4., dtype=torch.float64),
  tensor(5., dtype=torch.float64),
  tensor(6., dtype=torch.float64),
  tensor(7., dtype=torch.float64),
  tensor(8., dtype=torch.float64),
  tensor(9., dtype=torch.float64),
  tensor(10., dtype=torch.float64),
  

In [16]:
import json

with open(os.path.join("./dataset", "train_labels.json"), "w") as f:
    json.dump(train_dict, f)
with open(os.path.join("./dataset", "valid_labels.json"), "w") as f:
    json.dump(valid_dict, f)
with open(os.path.join("./dataset", "test_labels.json"), "w") as f:
    json.dump(test_dict, f)

TypeError: ignored